In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import os
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import shutil
from torchvision import transforms
from tqdm import tqdm

In [3]:
import kagglehub
path = kagglehub.dataset_download("csafrit2/plant-leaves-for-image-classification")

print("Path to dataset files:", path)

100%|██████████| 6.56G/6.56G [01:12<00:00, 97.3MB/s]

Extracting model files...


Path to dataset files: /root/.cache/kagglehub/datasets/csafrit2/plant-leaves-for-image-classification/versions/2


In [4]:

healthy = [
    os.path.join(path, 'Plants_2/train/Pomegranate healthy (P9a)'),
    os.path.join(path, 'Plants_2/train/Arjun healthy (P1b)'),
    os.path.join(path, 'Plants_2/train/Jamun healthy (P5a)')
]

diseased = [
    os.path.join(path, 'Plants_2/train/Pomegranate diseased (P9b)'),
    os.path.join(path, 'Plants_2/train/Arjun diseased (P1a)'),
    os.path.join(path, 'Plants_2/train/Jamun diseased (P5b)')
]

healthy_files = [os.path.join(h, f) for h in healthy for f in os.listdir(h) if os.path.isfile(os.path.join(h, f))]
diseased_files = [os.path.join(d, f) for d in diseased for f in os.listdir(d) if os.path.isfile(os.path.join(d, f))]



In [105]:

all_files = healthy_files + diseased_files
labels = [0] * len(healthy_files) + [1] * len(diseased_files)

print(f"Number of healthy images: {len(healthy_files)}")
print(f"Number of diseased images: {len(diseased_files)}")
print(f"Total number of images: {len(all_files)}")

train_files, test_files, train_labels, test_labels = train_test_split(all_files, labels, test_size=0.3, stratify=labels, random_state=42)


class PlantImageDataset(Dataset):
    def __init__(self, image_files, labels, transform=None, target_size=(128, 128)):
        self.image_files = image_files
        self.labels = labels
        self.target_size = target_size

        self.transform = transform if transform else transforms.Compose([
            transforms.Resize(self.target_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_files)


    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]

        return image, label
train_dataset = PlantImageDataset(train_files, train_labels, target_size=(128, 128))
test_dataset = PlantImageDataset(test_files, test_labels, target_size=(128, 128))


Number of healthy images: 755
Number of diseased images: 818
Total number of images: 1573


In [106]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [107]:
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn


CNN_model = models.resnet18(pretrained=True)
for param in CNN_model.parameters():
    param.requires_grad = False

num_features = CNN_model.fc.in_features
CNN_model.fc = nn.Linear(num_features, 2)

CNN_model = CNN_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(CNN_model.fc.parameters(), lr=0.001, momentum=0.9)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [108]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

def calculate_accuracy(predictions, labels):
    _, predicted = torch.max(predictions, 1)
    correct = (predicted == labels).sum().item()
    return correct / len(labels)


num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CNN_model = CNN_model.to(device)

for epoch in range(num_epochs):
    CNN_model.train()
    running_loss = 0.0
    running_accuracy = 0.0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = CNN_model(images)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_accuracy += calculate_accuracy(outputs, labels)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = running_accuracy / len(train_loader)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")



100%|██████████| 35/35 [01:00<00:00,  1.72s/it]


Epoch [1/10], Loss: 0.5642, Accuracy: 0.7192


100%|██████████| 35/35 [01:04<00:00,  1.83s/it]


Epoch [2/10], Loss: 0.3496, Accuracy: 0.8603


100%|██████████| 35/35 [01:03<00:00,  1.82s/it]


...

Epoch [8/10], Loss: 0.2153, Accuracy: 0.9130


100%|██████████| 35/35 [01:03<00:00,  1.82s/it]


Epoch [9/10], Loss: 0.2135, Accuracy: 0.9232


100%|██████████| 35/35 [01:02<00:00,  1.79s/it]

Epoch [10/10], Loss: 0.2028, Accuracy: 0.9255


In [109]:

CNN_model.eval()  # Set the model to evaluation mode
test_loss = 0.0
test_accuracy = 0.0

# Initialize counters for healthy vs diseased
healthy_count = 0
diseased_count = 0
correct_healthy = 0
correct_diseased = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get model outputs
        outputs = CNN_model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        test_accuracy += calculate_accuracy(outputs, labels)

        # Get predicted class
        _, predicted = torch.max(outputs, 1)

        # Count healthy and diseased predictions
        for i in range(len(labels)):
            if labels[i].item() == 0:  # Assuming label 0 is 'healthy'
                healthy_count += 1
                if predicted[i].item() == 0:
                    correct_healthy += 1
            elif labels[i].item() == 1:  # Assuming label 1 is 'diseased'
                diseased_count += 1
                if predicted[i].item() == 1:
                    correct_diseased += 1

# Calculate final metrics
test_loss /= len(test_loader)
test_accuracy /= len(test_loader)

print(f"Validation Loss: {test_loss:.4f}, Validation Accuracy: {test_accuracy:.4f}")
print(f"Healthy images: {healthy_count}, Diseased images: {diseased_count}")
print(f"Correct healthy predictions: {correct_healthy}")
print(f"Correct diseased predictions: {correct_diseased}")
print(f"Accuracy on healthy images: {correct_healthy / healthy_count:.4f}")
print(f"Accuracy on diseased images: {correct_diseased / diseased_count:.4f}")
print("Training completed!")

Validation Loss: 0.2097, Validation Accuracy: 0.9243
Healthy images: 227, Diseased images: 245
Correct healthy predictions: 214
Correct diseased predictions: 222
Accuracy on healthy images: 0.9427
Accuracy on diseased images: 0.9061
Training completed!


In [110]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [120]:
os.makedirs('/content/drive/My Drive/models', exist_ok=True)
model_path = '/content/drive/My Drive/models/cnn_model_weights.pth'
torch.save(CNN_model.state_dict(), model_path)


In [61]:
from PIL import Image
from torchvision import transforms
import torch

# Define the transformations (same as in the dataset)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Match target size used in dataset
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Match training normalization
])

# Load the image (replace with your new image path)
image_path = '/content/healthy.png'
image = Image.open(image_path).convert('RGB')  # Ensure it's RGB

# Apply the transformations
image = transform(image)

# Add batch dimension (model expects a batch, even for a single image)
image = image.unsqueeze(0)  # shape will be (1, 3, 128, 128)


# Move the model and image to the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_model.to(device)
image = image.to(device)

# Make the prediction
with torch.no_grad():  # No need to track gradients for inference
    output = CNN_model(image)

# Get predicted class
_, predicted_class = torch.max(output, 1)

# Map class index to class name
class_names = ['Healthy', 'Diseased']  # Modify if you have different classes
predicted_label = class_names[predicted_class.item()]

# Print the prediction
print(f"Predicted label: {predicted_label}")

Predicted label: Healthy


In [121]:
import sys
import shutil
import os
import numpy as np
from PIL import Image
import torch
# Import the CNN model from setup.py
# Instantiate the model and set it to evaluation mode


# Function to standardize the image
def standardize_image(image_path):
    image = Image.open(image_path).convert("RGB") #changes the color of the image
    img_array = np.array(image)
    mean = np.mean(img_array, axis=(0, 1))
    std = np.std(img_array, axis=(0, 1))
    standardized_img = (img_array - mean) / std
    standardized_img = np.clip(standardized_img, 0, 1) #smooths image over 0 to 1
    standardized_img = Image.fromarray((standardized_img * 255).astype(np.uint8))
    return standardized_img
#makes sure he image is standardized (in color, size, resolution )
# Function to classify the standardized image
def classify_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform_image_for_model(image)  # Transform for model input
    with torch.no_grad():
        output = CNN_model(image_tensor)  # Correct way to call the model
        _, predicted = torch.max(output, 1)
        label = "Healthy" if predicted.item() == 0 else "Unhealthy"
    return label

# Preprocessing: Convert the PIL image to a tensor for the model
def transform_image_for_model(image):
    image = image.resize((128, 128))  # Adjust size as required by the model
    image = np.array(image).transpose((2, 0, 1))  # Rearrange to CxHxW
    image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0) / 255.0  # Add batch dimension
    return image_tensor


# Function to handle file upload and processing
# Function to handle file upload and processing
def upload_file(file_path, target_directory):
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    target_path = os.path.join(target_directory, os.path.basename(file_path))

    # Check if source and destination are the same
    if os.path.abspath(file_path) == os.path.abspath(target_path):
        print(f"Source and target paths are the same: {file_path}")
    else:
        shutil.copy(file_path, target_path)  # Use copy instead of move if needed

    # Standardize and classify
    #the restandardization is needed to be properly classified
    standardized_image = standardize_image(target_path)
    standardized_image.save(target_path)
    print(f"Image standardized and saved to {target_path}")

    # Classify image
    classification = classify_image(target_path)
    #since this is a binary model classification is either healthy or unhealthy
    print(f"Image classified as: {classification}")

upload_file('/content/unheal.jpg', '/content/target')




Image standardized and saved to /content/target/unheal.jpg
Image classified as: Unhealthy
